## Initialization

In [ ]:
import matplotlib.pyplot as plt
from mog_experiment import *
%autoreload 2

In [ ]:
X_DIM = 3
NUM_DISTRIBUTIONS = 3

MEANS = np.zeros((X_DIM, X_DIM), dtype=np.float32)
CENTERS = np.sqrt(2)  # center of Gaussian
for i in range(NUM_DISTRIBUTIONS):
    MEANS[i::NUM_DISTRIBUTIONS, i] = CENTERS

params = {                          # default parameter values
    'x_dim': X_DIM,
    'num_distributions': NUM_DISTRIBUTIONS,
    'means': MEANS,
    'sigma': 0.05,
    'small_pi': 2E-16,
    'scale': 0.1,
    'num_samples': 200,
    'lr_init': 1e-3,
    'lr_decay_steps': 1000,
    'eps': 0.3,
    'temp_init': 15,
    'annealing_steps': 150,
    'annealing_rate': 0.98,
    'train_trajectory_length': 15,
    'test_trajectory_length': 2000,
    'num_training_steps': 30000,
    'tunneling_rate_steps': 200,
    'save_steps': 2500,
    'logging_steps': 100
}

## Build / Train model

In [ ]:
tf.reset_default_graph()

In [ ]:
#%pdb
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
model = GaussianMixtureModel(params, 
                             config=config,
                             log_dir='../log_mog_tf/run87/')
                             #log_dir='../log_mog_tf/run64/')

In [ ]:
model.sess.run(tf.global_variables_initializer())

In [ ]:
model.tunneling_rates_highT.keys()

In [ ]:
for key in model.tunneling_rates_highT.keys():
    #print(f'Step num: {key[0]}')
    #print(f'Temp: {key[1]}')
    model.steps_arr.append(key[0])
    model.temp_arr.append(key[1])
    #model.temp_arr.append(key[1])

In [ ]:
for idx in range(len(model.steps_arr)):
    model.steps_arr[idx] += 1

In [ ]:
model.steps_arr

In [ ]:
for key, val in model.tunneling_rates.items():
    model.tunneling_rates_avg.append(val[0])
    model.tunneling_rates_err.append(val[1])
for key, val in model.tunneling_rates_highT.items():
    model.tunneling_rates_avg_highT.append(val[0])
    model.tunneling_rates_err_highT.append(val[1])
    
for key, val in model.acceptance_rates.items():
    model.acceptance_rates_avg.append(val[0])
    model.acceptance_rates_err.append(val[1])
for key, val in model.acceptance_rates_highT.items():
    model.acceptance_rates_avg_highT.append(val[0])
    model.acceptance_rates_err_highT.append(val[1])
    
for key, val in model.distances.items():
    model.distances_avg.append(val[0])
    model.distances_err.append(val[1])
for key, val in model.distances_highT.items():
    model.distances_avg_highT.append(val[0])
    model.distances_err_highT.append(val[1])

In [ ]:
model._save_variables()

In [ ]:
model._init_params(params)

In [ ]:
model._load_variables()

In [ ]:
model.steps_arr

In [ ]:
model.tunneling_rates

In [ ]:
import os

In [ ]:
model.__dict__.keys()

In [ ]:
get_vals_as_arr = lambda _dict: np.array(list(_dict.values()))
tr = get_vals_as_arr(model.tunneling_rates)
ar = get_vals_as_arr(model.acceptance_rates)
dr = get_vals_as_arr(model.distances)
#np.array(list(model.tunneling_rates.values()))

In [ ]:
for key in model.attrs_dict.keys():
    in_file = model.info_dir + key + '.npy'
    if os.path.isfile(in_file):
        setattr(model, key, np.load(in_file))
        print(f'Set model.{key} to values read in from: {in_file}')
        #print(model.key == np.load(in_file))

In [ ]:
model.sess.run(tf.global_variables_initializer())

In [ ]:
attrs_dict = model.attrs_dict

In [ ]:
attrs_dict['steps_arr']

In [ ]:
model.train(params['num_training_steps'], plot=True)

## OLD

In [ ]:
#trajectories, loss_arr, px_arr = model.generate_trajectories(num_samples=100, num_steps=100, temperature=1.)

#config = tf.ConfigProto(log_device_placement=True)
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#model.build_graph()

In [ ]:
def calc_avg_distance1(trajectories):
    distances_arr = []
    for trajectory in trajectories:
        distance_arr = []
        for idx in range(1, len(trajectory)):
            diff = trajectory[idx] - trajectory[idx-1]
            dist = np.sqrt(np.dot(diff, diff.T))
            distance_arr.append(dist)
        distances_arr.append(sum(distance_arr))
    return np.mean(distances_arr)

def calc_avg_distance2(trajectories):
    dist = lambda d: np.sqrt(np.dot(d, d.T))
    #distances_arr = np.mean([[dist(d) for d in [t[:-1, :] - t[1:, :] for t in trajectories]]
    #for trajectory in trajectories:
        diff = trajectory[:-1, :] - trajectory[1:, :]
        distance = sum([np.sqrt(np.dot(d, d.T)) for d in diff])
        distances_arr.append(distance)
    return np.mean(distances_arr)

In [ ]:
%timeit avg_dist = avg_distance_traveled(trajectories.transpose([1,0,2]))
#print(avg_dist)

%timeit avg_dist1 = calc_avg_distance1(trajectories.transpose([1,0,2]))
#print(avg_dist1)

%timeit avg_dist2 = calc_avg_distance2(trajectories.transpose([1,0,2]))
#print(avg_dist2)

distances = np.array([np.sqrt(np.dot(disp, disp.T)) for disp in displacements])

In [ ]:
model.train(params['num_training_steps'], config=config, plot=True)

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
model.build_graph()
model.train(params['num_training_steps'], config=config, plot=False)